# Titanic Competition (https://www.kaggle.com/competitions/titanic)


Titanic sank on its maiden voyage from England to New York in 1912. Many people died in the tragety. This competition is aimed to predict the survivals of people as accurate as possible based on their information (age, gender, fare, cabin, etc). Please visit https://www.kaggle.com/competitions/titanic fore more information.

## General workflow of developing machine learning solutions

1. Question or problem definition.
2. Acquire training and testing data.
3. Data exploration
4. Data preprocessing (handling missing data and outliers, encoding categorical features, feature selection and engineering)
5. Train and evaluate various models on cleaned data.
6. Repeat steps 3-5 to get best results
7. Deploy the best trained model (Submit the results obtained by running the model on testing data)

## Sklearn classifiers
https://scikit-learn.org/stable/supervised_learning.html

# Import required packages

In [ ]:
# Data analysis and preprocessing
import pandas as pd
import numpy as np
import random as rnd
from sklearn import preprocessing

# Visualization
#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning (https://scikit-learn.org/stable/supervised_learning.html)
from sklearn.linear_model import LogisticRegression # linear model + sigmoid function
from sklearn.svm import SVC, LinearSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


# Load data

The data provided from the Kaggle Titanic Competition can be loaded as Pandas' dataframes

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Quickly examine the data

Please find description of the data at https://www.kaggle.com/c/titanic/data.

In [ ]:
# To see what features are available in the dataset
print(train_df.shape)
print(train_df.columns.values)
print('-'*80)
print(test_df.shape)
print(test_df.columns.values)

In [ ]:
# To display the first n=7 rows of the dataframe
train_df.head(5)
#print('-'*80)
#train_df.tail()

In [ ]:
train_df.info()
print('-'*80)
test_df.info()

In [ ]:
# Find the row with NaN in Fare
test_df[test_df.Fare.isnull()]


**What we've learned**
- The training dataset has 12 columns while the testing dataset has 11 columns. The testing dataset has one column (survived) missing, which is meant for a trained model to predict.
- Some columns are numeric features (int64, float64), others are strings (as numpy objects) which may be interpreted as categorical features later on.
- Some columns (*Age, Cabin, Embarked*) have missing values (marked as NaN). There is only one missing value for *Fare* in testing dataset.

In [ ]:
# By default, it only shows numeric columns
train_df.describe()

**What we've learned**

- These numeric features are ready to be used in training
- PassengerId is not useful in predicting survival

In [ ]:
# To show those non-numeric columns
train_df.describe(include='all')

In [ ]:
train_df.hist(figsize=(15,15))

**What we've learned**

- Names are unique across the dataset (count=unique=891)
- **Sex** has two possible values with 65% male (top=male, freq=577/count=891).
- Cabin values have several dupicates across samples. This is probably because multiple passengers shared a cabin.
- **Embarked** has three possible values. S port used by most passengers (top=S)
- Ticket feature has high ratio (22%) of duplicate values (unique=681).

# Evaluate feature importance

We evaluate feature importance based on its correlation with the target feature (survived).

In [ ]:
corr = train_df.corr(numeric_only=True)
corr.style.background_gradient(cmap='coolwarm')

The above correlation map only shows the correlations between numeric features.
Now let's examine the importance of discrete features (int64, object) that have limited number of values.

In [ ]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

**Features to keep:**
- Numeric
    - Pclass
    - Fare
    - Parch
    - Age
    - SibSp
- Categorical
    - Sex
    - Embarked
    
**Features to discard**
- Numeric
    - PassengerId
- Categorical
    - Name
    - Ticket
    - Cabin

# Data preparation

## 1. Drop irrelevant or less important features

In [ ]:
train_df1 = train_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_df1 = test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
print(train_df1.shape, test_df1.shape)

## 2 Handle missing values (NaN)
Options:
- Remove all rows that contain missing values: df.dropna(axis=0)
- Remove all columns that contain missing values: df.dropna(axis=1)
- **Imputate missing values**
    - Categorical feature: 
        - Replaced with a new unique value (such as "Unknown")
        - Replaced with the most frequently occuring one
    - Numeric feature:
        - Replace with median (or mean or mode) of the dataset
        - Estimate a more reasonable value for the missing value based on other available features

In [ ]:
train_df2 = train_df1.copy(deep=True)
test_df2 = test_df1.copy(deep=True)

# Age and Fare
age_median = train_df2.Age.median()
fare_median = train_df2.Fare.median()
print(age_median, fare_median)

train_df2["Age"] = train_df2["Age"].fillna(age_median)
train_df2["Fare"] = train_df2["Fare"].fillna(fare_median)
train_df2["Embarked"] = train_df2["Embarked"].fillna('Unknown')

print(train_df2.Embarked.unique())

test_df2["Age"] = test_df2["Age"].fillna(age_median)
test_df2["Fare"] = test_df1["Fare"].fillna(fare_median)
test_df2["Embarked"] = test_df2["Embarked"].fillna('Unknown')

## 3. Convert a categorical features to numerical ones

Two Categorical features to convert: *Sex, Embarked*

In [ ]:
train_df3 = train_df2.copy(deep=True)
test_df3 = test_df2.copy(deep=True)

train_df3['Sex'] = train_df3['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
test_df3['Sex'] = test_df3['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# Use either mapping or ordinal encoder
#train_df3['Embarked'] = train_df3['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2, 'Unknown': 3} ).astype(int)
#test_df3['Embarked'] = test_df3['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2, 'Unknown': 3} ).astype(int)

enc = preprocessing.OrdinalEncoder(dtype=int)
enc.fit(train_df3[['Embarked']])
train_df3['Embarked'] = enc.transform(train_df3[['Embarked']])
test_df3['Embarked'] = enc.transform(test_df3[['Embarked']])

train_df3.info()
print(train_df3.Sex.unique())
print(train_df3.Embarked.unique())

## 4. Estimate feature importance based on correlation again

In [ ]:
corr = train_df3.corr()
corr.style.background_gradient(cmap='coolwarm')

Features in the desceasing order of importance:
- Sex (0.543351)
- Pclass (-0.338481)
- Fare (0.257307)
- Embarked (0.118026)
- Parch (0.0816294)
- age (-0.0816294)
- SibSp (-0.0353225)

# Train prediction models

## 1. Organize data in pairs of X (features), Y (target)

In [ ]:
X_train = train_df3.drop("Survived", axis=1, inplace=False)
Y_train = train_df3["Survived"]
X_test = test_df3.copy()
X_train.shape, Y_train.shape, X_test.shape

## 2. Train and evaluate various models
### 2.1 Train and evaluate a single model

In [ ]:
# Choose a classification model
classifier = SVC(gamma='auto') # gamma='scale'

In [ ]:
# Train and evaluate model with the same dataset (bad)
classifier.fit(X_train, Y_train)
print('Accuracy using SVC:', classifier.score(X_train, Y_train) * 100)

In [ ]:
# Train and evaluate model with the different dataset (ok)
train_x, test_x, train_y, test_y = train_test_split(X_train, Y_train, test_size = 0.25)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
classifier.fit(train_x, train_y)
print('Accuracy using SVC:', classifier.score(test_x, test_y) * 100)

In [ ]:
# n-fold cross validation (better)
acc = cross_val_score(classifier, X_train, Y_train, cv=4)
print(acc, acc.mean())

### 2.2 Cross validate with more classifiers

In [ ]:
logreg = LogisticRegression() #(max_iter=200)
linear_svc = LinearSVC() #(dual='auto')
svc = SVC()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
gboost = HistGradientBoostingClassifier()
knn = KNeighborsClassifier(n_neighbors=3)
gaussian = GaussianNB()

models = {
    'Logistic Regression' : logreg, 
    'Linear SVC' : linear_svc, 
    'SVC' : svc, 
    'Decision Tree' : decision_tree,
    'Random Forest' : random_forest, 
    'HistGradientBoosting' : gboost,
    'KNN' : knn, 
    'Naive Bayes' : gaussian
}

In [ ]:
def cross_val_all_models(models, X_train, Y_train, n_folds=4) :
    for name, classifier in models.items() :
        acc = cross_val_score(classifier, X_train, Y_train, cv=n_folds)
        print(name, acc, acc.mean())

In [ ]:
cross_val_all_models(models, X_train, Y_train)

### 2.3 Choose the model with the highest accuracy and re-train it with the entire training set

In [ ]:
gboost.fit(X_train, Y_train)
Y_test = gboost.predict(X_test)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_test
    })
print(submission.head())

submission.to_csv('submission.csv', index=False)

### 2.4 Check the accuracy
- By submitting "submission.csv" on https://www.kaggle.com/competitions/titanic
- Compare the predictions with the leaked groundtruth

In [ ]:
def CalculateAccuracy(prediction) :
    gt_df = pd.read_csv('leaked-titanic.csv')
    gt = gt_df["Survived"].values
    #print(gt)
    return (np.mean(prediction==gt))

In [ ]:
print("Accuracy evaluated on testing dataset:", CalculateAccuracy(Y_test))

# Improve prediction accuracy
## Better data preparation
### 1. Dig useful information out of Name

In [ ]:
train_df.Name.head(25)

In [ ]:
train_df4 = train_df3.copy(deep=True)
test_df4 = test_df3.copy(deep=True)

train_df4['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
print(train_df4['Title'].describe())
print('-'*80)
test_df4['Title'] = test_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
print(test_df4['Title'].describe())

In [ ]:
print(train_df4['Title'].value_counts())
print('-'*80)
print(test_df4['Title'].value_counts())

In [ ]:
# Feature values that have limited appearances cannot provide stable stastistic information
# to learn. So, we tag them with a new value "rare"
train_df4['Title'] = train_df4['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

train_df4['Title'] = train_df4['Title'].replace('Mlle', 'Miss')
train_df4['Title'] = train_df4['Title'].replace('Ms', 'Miss')
train_df4['Title'] = train_df4['Title'].replace('Mme', 'Mrs')

test_df4['Title'] = test_df4['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

test_df4['Title'] = test_df4['Title'].replace('Mlle', 'Miss')
test_df4['Title'] = test_df4['Title'].replace('Ms', 'Miss')
test_df4['Title'] = test_df4['Title'].replace('Mme', 'Mrs')


In [ ]:
print(train_df4['Title'].value_counts())
print('-'*80)
print(test_df4['Title'].value_counts())

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

train_df4['Title'] = train_df4['Title'].map(title_mapping)
test_df4['Title'] = test_df4['Title'].map(title_mapping)

train_df4.describe()

In [ ]:
train_df4.info()

### 2. Create new feature "family size" or "travel alone"

FamilySize = Parch + SibSp. 
Travel alone =  true if FamilySize=0, or false otherwise

In [ ]:
train_df5 = train_df4.copy(deep=True)
test_df5 = test_df4.copy(deep=True)

train_df5['FamilySize'] = train_df5['SibSp'] + train_df5['Parch']
test_df5['FamilySize'] = test_df5['SibSp'] + train_df5['Parch']

train_df5[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df5['IsAalone'] = (train_df5['FamilySize']==0).astype(int)
test_df5['IsAalone'] = (test_df5['FamilySize']==0).astype(int)

train_df5[['IsAalone', 'Survived']].groupby(['IsAalone'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Let us drop Parch, SibSp, and FamilySize features in favor of IsAlone.

In [ ]:
# Keep or discard feature "FamilySize"
train_df5 = train_df5.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df5 = test_df5.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)

### 3 One-hot encoding "Embarked" instead of ordinal encoding

#### One-hot encoding (to make it take effect, change this cell from Markdown to Code)
enc = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(train_df4[['Title']])

train_oh_df = pd.DataFrame(enc.transform(train_df4[['Title']]))
train_oh_df.index = train_df4.index
train_oh_df = train_oh_df.add_prefix('Title_')
test_oh_df = pd.DataFrame(enc.transform(test_df4[['Title']]))
test_oh_df.index = test_df4.index
test_oh_df = test_oh_df.add_prefix('Title_')

train_df5 = train_df5.drop(['Title'], axis=1)
test_df5 = test_df5.drop(['Title'], axis=1)

train_df5 = pd.concat([train_df5, train_oh_df], axis=1)
test_df5 = pd.concat([test_df5, test_oh_df], axis=1)

train_df5.describe()

### 4 One-hot encoding "Cabin"

enc = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse_output=False) 
enc.fit(train_df[['Cabin']])

train_oh_df = pd.DataFrame(enc.transform(train_df[['Cabin']])) 
train_oh_df.index = train_df5.index 
train_oh_df = train_oh_df.add_prefix('Cabin_') 

test_oh_df = pd.DataFrame(enc.transform(test_df[['Cabin']])) 
test_oh_df.index = test_df5.index 
test_oh_df = test_oh_df.add_prefix('Cabin_')

train_df5 = pd.concat([train_df5, train_oh_df], axis=1) 
test_df5 = pd.concat([test_df5, test_oh_df], axis=1)

train_df5.describe()

## Quick cross validate models with the better prepared data 

In [ ]:
X_train = train_df5.drop("Survived", axis=1, inplace=False)
Y_train = train_df5["Survived"]
X_test = test_df5.copy()

In [ ]:
cross_val_all_models(models, X_train, Y_train)

## Tune hyper-parameters of models
### Manual tuning

In [ ]:
logreg = LogisticRegression(max_iter=500)
linear_svc = LinearSVC(dual='auto', tol=1e-5)
svc = SVC(C=500.0, degree=2)
#decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier(n_estimators=100, max_depth=7)
gboost = HistGradientBoostingClassifier(max_depth=7)
knn = KNeighborsClassifier(n_neighbors=8)
gaussian = GaussianNB()

models = {
    'Logistic Regression' : logreg, 
    'Linear SVC' : linear_svc, 
    'SVC' : svc, 
    'Decision Tree' : decision_tree,
    'Random Forest' : random_forest, 
    'HistGradientBoosting' : gboost,
    'KNN' : knn, 
    'Naive Bayes' : gaussian
}

In [ ]:
cross_val_all_models(models, X_train, Y_train)

### Grid-search tuning top 3 classifiers
#### Random forest

In [ ]:
# Random Forest
parameters = {'n_estimators':[120, 150, 180], 'max_depth':[7, 9, 11]}
gs = GridSearchCV(random_forest, parameters, cv=4)
gs.fit(X_train, Y_train)

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
Y_test1 = gs.best_estimator_.predict(X_test)
print("Accuracy evaluated on testing dataset:", CalculateAccuracy(Y_test1))

#### Histogram Gradient Boosting 

In [ ]:
parameters = {'max_depth':[7, 9, 11], 'l2_regularization':[0.3, 0.5], 'max_iter':[70], 'max_leaf_nodes':[20, 35, 50]}
gs = GridSearchCV(gboost, parameters, cv=4)
gs.fit(X_train, Y_train)

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
Y_test2 = gs.best_estimator_.predict(X_test)
print("Accuracy evaluated on testing dataset:", CalculateAccuracy(Y_test2))

#### SVC

In [ ]:
svc = SVC(C=500.0, degree=2)
parameters = {'C':[400.0, 500.0, 600.0], 'degree':[2, 3, 4]}
gs = GridSearchCV(svc, parameters, cv=4)
gs.fit(X_train, Y_train)

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
Y_test3 = gs.best_estimator_.predict(X_test)
print("Accuracy evaluated on testing dataset:", CalculateAccuracy(Y_test3))

## Collective voting (does not improve anything)

In [ ]:
Y_test = Y_test1 + Y_test2 + Y_test3
Y_test = (Y_test>1).astype(int)
print("collective voting accuracy:", CalculateAccuracy(Y_test))

## Possible ways to further improve the results
* Treat categorical features as 'category' type
* Use One-hot encoded 'Cabin' feature
* Feature engineering
    * Binning
    * Scaling (min-max or standard)
    * PCA
* Grid-search for more hyper-parameters 
* Try other models
    * Catboost
    * XGBoost
